In [1]:
import numpy as np
import JupyterNotebooksLib as slicernb
import slicer
import vtk

In [2]:
""""
Converts a string containing the anybody definition of a matrix and converts it to an numpy.array
"""    
def AnyMatrix2Array(string):
    import numpy as np
    string = string.replace(" ","")
    matrix = []
    if string[0:2]=="{{":
        Type = "Matrix"
        rows = string.split("},{")
        rows[0] = rows[0].replace("{{","")
        rows[-1] = rows[-1].replace("}}","")
    else:
        
        rows = string.split("},{")
        rows[0] = rows[0].replace("{","")
        rows[-1] = rows[-1].replace("}","")
        
        if len(rows[0].split(","))==1:
            Type = "Value"
        else:
            Type = "Vector"
    
    
    
    if Type == "Value":
        matrix = eval(rows[0])
    elif Type == "Vector":
        rows = rows[0].split(",")
        rows = [eval(i) for i in rows]
        matrix.append(rows)
        matrix = np.array(matrix)
    else:  
        for row in rows:
            elements = row.split(",")
            elements = [eval(i) for i in elements]
            matrix.append(elements)
        matrix = np.array(matrix)

    return matrix

"""
Input : rotation matrix and translation vector to a 4x4 matrix for a 3d slicer transform
"""
def TransformMatrix(RotMat=None,Vect=None):
    import numpy as np
    
    if RotMat is None:
        RotMat = np.eye(3)
    if Vect is None:
        Vect = np.zeros(3)
    
    Mat = np.eye(4)
    
    Mat[0:3,0:3]= RotMat
    Mat[0:3,3] = Vect
    return Mat


"""
Function to set the 4x4 transform matrix to a transform in 3d slicer from arrays
"""
def SetTransformMatrix(TransformName, RotMat=False,Vect=False):
    
    TransformMat = TransformMatrix(RotMat,Vect)
    
    Transform = slicer.util.getNode(TransformName)
    Transform.SetMatrixTransformToParent(slicer.util.vtkMatrixFromArray(TransformMat))


In [3]:
def AngleBetweenAxis(AxisNodeNames):
  import numpy as np
  lineDirectionVectors = []
  for lineNodeName in AxisNodeNames:
    lineNode = slicer.util.getFirstNodeByClassByName("vtkMRMLMarkupsLineNode", lineNodeName)
    lineStartPos = np.zeros(3)
    lineEndPos = np.zeros(3)
    lineNode.GetNthControlPointPositionWorld(0, lineStartPos)
    lineNode.GetNthControlPointPositionWorld(1, lineEndPos)
    lineDirectionVector = (lineEndPos-lineStartPos)/np.linalg.norm(lineEndPos-lineStartPos)
    lineDirectionVectors.append(lineDirectionVector)
  angleRad = vtk.vtkMath.AngleBetweenVectors(lineDirectionVectors[0], lineDirectionVectors[1])
  angleDeg = vtk.vtkMath.DegreesFromRadians(angleRad)
  return angleDeg  
    


In [6]:
# local tilt axis

RHum_Me = AnyMatrix2Array("{{0.8296721, -0.4502591, -0.3300165},{-0.3033696, -0.8599025, 0.4105297},{-0.4686267, -0.2404881, -0.8500321}}")
RGlen_Me = AnyMatrix2Array("{{0.8216931, 0.271819, 0.5009341},{-0.315746, 0.9488388, 0.003061939},{-0.4744734, -0.1606839, 0.86548}}")

SetTransformMatrix("Hum_Me",RotMat = RHum_Me)
SetTransformMatrix("Glen_Me",RotMat = RGlen_Me)

Angle = AngleBetweenAxis(["x Hum Me","x Glen Me"])
print(Angle)

0.9077880669465732


In [ ]:
# scapula axis
# xup


RHum_Me = AnyMatrix2Array("{{0.8296721, -0.4502591, -0.3300165},{-0.3033696, -0.8599025, 0.4105297},{-0.4686267, -0.2404881, -0.8500321}}")
RGlen_Me = AnyMatrix2Array("{{0.8216931, 0.271819, 0.5009341},{-0.315746, 0.9488388, 0.003061939},{-0.4744734, -0.1606839, 0.86548}}")

SetTransformMatrix("Hum_Me",RotMat = RHum_Me)
SetTransformMatrix("Glen_Me",RotMat = RGlen_Me)

Angle = AngleBetweenAxis(["x Hum Me","x Glen Me"])
print(Angle)